# Python 操作数据库

In [3]:
import sqlite3

sqlite_data_path = './tysql.sqlite'

In [4]:
# Python 操作数据库一般步骤
conn = sqlite3.connect(sqlite_data_path)  # 连接 sqlite
cur = conn.cursor()  # 创建游标
sql = 'SELECT * FROM Customers LIMIT 2'  # 要执行的 SQL 语句
cur.execute(sql)  # SQL 语句
conn.commit()  # 提交
cur.close()  # 关闭游标
conn.close()  # 关闭连接

In [9]:
# pandas 读取数据库
import pandas as pd

sql = '''
SELECT * 
FROM Customers 
LIMIT 2;
'''

df = pd.read_sql(sql=sql, con=sqlite3.connect(sqlite_data_path))
df

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000001,Village Toys,200 Maple Lane,Detroit,MI,44444,USA,John Smith,sales@villagetoys.com
1,1000000002,Kids Place,333 South Lake Drive,Columbus,OH,43333,USA,Michelle Green,None


# SELECT 查询  

以下顺序必须保持。

SELECT DISTINCT column1, column2 -- 列  
FROM table -- 表  
WHERE -- 过滤  
GROUP BY -- 分组  
HAVING -- 过滤分组  
ORDER BY -- 排序  
LIMIT 返回多少行 ;  

完全限定列名：table.column ，跨表查询时会用到。

## 1、基础检索

SELECT  
FROM  
LIMIT ;  

### 检索单个列

SELECT prod_name  
FROM Products ;  

### 检索多个列

SELECT prod_id, vend_id, prod_name  
FROM Products ;  

### 检索所有列

SELECT *  
FROM Products ;

### DISTINCT 检索唯一值  
DISTINCT 返回所有列的唯一值，不能部分返回

SELECT DISTINCT prod_id, prod_name  
FROM Products ;

### LIMIT 限制返回结果
SQL 行号从 0 开始  

SELECT prod_id, prod_name  
FROM Products  
LIMIT 5 ;  -- 返回前 5 行  

#### LIMIT OFFSET  

SELECT prod_id, prod_name  
FROM Products  
LIMIT 5 OFFSET 3 ;  -- 从第 5 行开始，返回 3 行，如果超出范围不会报错，可以简写：LIMIT 5, 3 ;  

### 注释  

-- 单行注释  

/\*  
多行注释  
\*/  
 

## 2、ORDER BY 排序    

ORDER BY 需要在除 LIMIT 外，所有 SQL 语句的最后一行。  
可以使用未检索的列进行排序。  

### 排序单个列

SELECT prod_id, prod_name  
FROM Products  
ORDER BY prod_price ; -- 可以使用未检索的列进行排序  

### 排序多个列（传递列名）

SELECT prod_id, prod_price, prod_name  
FROM Products  
ORDER BY prod_price, prod_name ; -- 先按价格，再按名称排序  

### 排序多个列（传递列相对位置）  

可以和传递列名混合使用。  

SELECT prod_id, prod_price, prod_name  
FROM Products  
ORDER BY 2, 3 ; --  相对位置从 1 开始。如果使用未检索的列排序，则不能传递相对位置  

### 指定排序方向  

DESC（降序，descending）、  ASC（默认，升序，ascending）

SELECT prod_id, prod_price, prod_name  
FROM Products  
ORDER BY prod_price DESE, prod_name ; -- 排序方向只对单个列有效，如果要全部指定，则需要逐个指定排序方向。  

## 3、WHERE 过滤  

WHERE 在数据分组前，根据条件过滤数据。  
可以使用为检索的列进行过滤。

### 所有过滤操作符  

| 操作符 | 说明 |
| ------ | ------ |
| = | 等于 |
| != | 不等于 |
| <> | 不等于 |
| < | 小于 |
| > | 大于 |
| <= | 小于等于 |
| >= | 大于等于 |
| ! | 不小于 |
| BERWEEN | 在指定两个值之间 |
| ISNULL | 空值 |
| NOTNULL | 非空值 |

SELECT prod_name, prod_price  
FROM Products  
WHERE prod_price = 3.49 ;  

SELECT prod_name, prod_price  
FROM Products  
WHERE prod_price BERWEEN 5 AND 10;  

SELECT prod_name, prod_price  
FORM Products  
WHERE prod_price ISNULL;  

## 4、高级过滤  

组合 WHERE 条件

### AND（且）、OR（或）组合 WHERE 条件  

AND 逻辑且，对多个列进行过滤。  
OR 逻辑或，一般第一个条件满足就不在计算后面的条件了。

SELECT prod_id, prod_price, prod_name  
FROM Products  
WHERE vend_id='DLL01' AND prod_price<=4;  

### 多个条件，需使用括号明确指定条件关系 

SELECT prod_id, prod_price, prod_name  
FROM Products  
WHERE prod_price>=10 AND (vend_id='DLL01' OR vend_id='BRS01')  

### IN  对一列，使用多个条件过滤  

SELECT prod_name, prod_price  
FROM Products  
WHERE vend_id IN ('DLL01', 'BRS01');  

也可用 OR 代替  

WHERE vend_id='DLL01' OR vend_id='BRS01';  

 可以包含其他 SELECT 语句  
 

### NOT 否定后续条件  

NOT 从不单独使用

SELECT prod_name  
FROM Products  
WHERE NOT vend_id='DLL01';  

## 5、使用通配符过滤  

主要是通配符、LIKE，匹配文本

### LIKE 匹配、通配搜索  

#### % 通配符，匹配一个或多个字符，不能匹配 NULL  
SELECT prod_id, prod_name  
FROM Products  
WHERE prod_name LIKE 'Fish%'; -- 匹配 Fish 开头的所有名称，注意：大小写敏感  

#### _ 通配符，只匹配单个字符  
SELECT prod_id, prod_name  
FROM Products  
WHERE prod_name LIKE 'F_sh b_an __g toy';  

#### [ ] 通配符，指定要通配的字符集。注意：MySQL 不支持，只有 Access 和 SQL Server 支持
SELECT prod_id, prod_name  
FROM Products  
WHERE prod_name LIKE '[RF]%'; -- 匹配 R 或 F 开头的名称  

## 6、计算字段  

很多时候，需要将多个列处理后再输出，比如：拼接、计算。  
还会涉及到 AS 关键字。

### 拼接字段  

Access、SQL Server：+  
SQLite：||  
MySQL：concat() 函数  

最基本的拼接：  
SELECT vend_name || vend_country  
FROM Vendors;  

整理一下格式：  
SELECT vend_name || ' (' || vend_country || ')'  
FROM Vendors;  

#### AS 关键字，添加别名，以便引用拼接的字段：  
SELECT (vend_name || ' (' || vend_country || ')') AS vend_title  
FROM Vendors;  


### 执行算术计算

比如计算订单号20008中每项物品的总价：  
SELECT prod_id, quantity, item_price, quantity*item_price AS expanded_price  
FROM OrderItems  
WHERE order_num=20008;   

SELECT 语句可以单独使用，执行简单操作，比如：  
SELECT 2 + 3;  
SELECT TRIM('  abc    '); -- 去掉左右的空格    
SELECT RTRIM('abc   '); -- 去掉右边的空格    
SELECT LTRIM('   abc'); -- 去掉左边的空格    

## 7、聚集函数  

有时我们想得到数据汇总的结果，并不需要数据本身，这时需要用到聚集函数。  

### 聚集函数列表  

| 函数 | 说明 |
| ------ | ------ |
| avg() | 返回平均值 |
| count() | 返回行数 |
| max() | 返回最大值 |
| min() | 返回最小值 |
| sum() | 返回求和结果 |

### avg() 函数  

avg() 作用于单个列，求多个列平均数需要用多个 avg() 函数。  

SELECT avg(prod_price) AS avg_price  
FROM Products  
WHERE vend_id='BRS01';  

### count() 函数

count(*)：返回整个表的行数，包含 NULL  
count(column)：返回指定列的行数，不包含 NULL  

SELECT count(*) AS num_product  
FROM Products;  

### max() 函数  

如果 max() 作用于文本，则返回排序后的最后一行。  

SELECT max(prod_price) AS max_price  
FROM Products;  

### min() 函数

如果 min() 作用于文本，则返回排序后的第一行。  

SELECT min(prod_price) AS min_price  
FROM Products;  

### sum() 函数  

SELECT sum(prod_price) AS sum_price  
FROM Products  
WHERE vend_id='BRS01';  

SELECT sum(item_price*order_item) AS sum_item  
FROM OrderItems;  

### 聚集函数的 ALL、DISTINCT 属性

ALL 属性，默认，计算所有值  
DISTINCT 属性，计算唯一值  

SELECT avg(DISTINCT prod_price) as avg_price  
FROM Products;  

### 组合聚集函数  

可以将多个聚集函数组合使用。

SELECT avg(DISTINCT prod_price) as avg_price, min(prod_price) AS min_price  
FROM Products;  


## 8、数据分组  

SELECT 的最后两个子句：GROUP BY 和 HAVING  



#### GROUP BY 分组

SELECT vend_id, count(*) AS num_prods  
FROM Products  
GROUP BY vend_id;  

#### HAVING 过滤分组  

HAVING 在数据分组后，过滤数据。

SELECT vend_id, count(*) AS num_prods  
FROM Products  
GROUP BY vend_id  
HAVING num_prods >= 3;  

WHERE 和 HAVING 同时使用：

SELECT vend_id, count(\*) AS num_prods  
FROM Products  
WHERE prod_price > 4  
GROUP BY vend_id  
HAVING count(\*) >= 2;    